Part 1 - Wikipedia scrape

In [1]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
print("Libraries imported.")

Libraries imported.


In [3]:
# Scrape the Wikipedia page of interest
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
print("Page scraped.")

Page scraped.


In [4]:
# Create column headers
headers = ["PostalCode", "Borough", "Neighborhood"]

# Apply headers
df.columns = headers

# Set the column labels to equal the values in the first row.
df.rename(columns = df.iloc[0])

# Drop the first row
df.drop(df.index[0], inplace=True)

In [ ]:
# Drop missing values in the Neighborhood column
df1 = df.dropna(subset=["Neighborhood"], axis=0)

# Replace / with ,
df1.replace(to_replace=" /", value=",", inplace=True, regex=True)

Part 1 DataFrame

In [14]:
# Reindex
df1.reset_index(drop=True, inplace=True)
df1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
# Shape
df1.shape

(103, 3)

Part 2 - Add Latitude and Longitude Coordinates.

In [12]:
# Read csv file
latlong = pd.read_csv('C:/Users/danbr/Desktop/Applied Data Science/Course 4/Week 3/Geospatial_Coordinates.csv')

Part 2 DataFrame

In [13]:
# Join dataframes
df2 = pd.merge(df1, latlong)
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Part 3 - Cluster the Neighborhoods in Toronto Based on Postal Code

In [1]:
!pip install geopy
print('geopy installed')
from geopy.geocoders import Nominatim
print('Nominatim imported')

geopy installed
Nominatim imported


In [3]:
# Get Toronto latitude and longitude
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


Each blue dot on the map indicates 1 of 103 neighborhoods.

In [21]:
# Create map of Toronto
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoop_start=1)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
   
map_toronto